In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import optimize

import constants as const

plt.style.use('science')

In [ ]:
lo = np.loadtxt('spectra_local_lo_a0_fix_hpp_n_14.txt')
nlo = np.loadtxt('spectra_local_nlo_a0_B6_fix_coord_hp_n_14_coord_ir_corrected.txt')
mm = np.loadtxt('spectra_Mott_Massey.txt')

In [ ]:
1/np.sqrt(2*const.MU*-mm)

In [ ]:
def convergence(lamb, c1, c2):
    return c1 * (1 + c2*(1/lamb))


def convergence_improved(lamb, c1, c2, q):
    return c1 * (1 + c2*(q/lamb))


def convergence2(lamb, c1, c2):
    return c1 * (1 + c2*(1/lamb)**2)


def convergence12(lamb, c1, c2, c3):
    return c1 * (1 + c2/lamb + c3/lamb**2)


def convergence12_improved(lamb, c1, c2, c3, q):
    return c1 * (1 + c2*(q/lamb) + c3*(q/lamb)**2)

In [ ]:
R_lo = lo[:, 0]
x_lo = const.BETA4 / R_lo
energies_lo = lo[:, 1:].T

ii = np.intersect1d(
    np.where(x_lo > 60)[0],
    np.where(x_lo < 70)[0]
)

fig, ax = plt.subplots(dpi=200)
fig.patch.set_facecolor('white')

lo_asymp = []

for (i, row) in enumerate(energies_lo):
    gamma = np.sqrt(2*const.MU*-row[-1])
    pars, cov = optimize.curve_fit(lambda x, c1, c2: convergence_improved(x, c1, c2, gamma), x_lo[ii], row[ii])
    print(pars)
    lo_asymp.append(pars[0])
    
    ax.plot(x_lo[ii], row[ii], color=f'C{i}')
    ax.plot(x_lo[ii], convergence_improved(x_lo[ii], *pars, gamma), linestyle='--', color=f'C{i}')
    ax.axhline(pars[0], color='C6')

## NLO Quadratic

In [ ]:
gamma_6 = np.sqrt(2*const.MU*-row[-1])

In [ ]:
lower_bounds = [
    -np.inf,
    -50,
    -100
]

upper_bounds = [
    np.inf,
    50,
    100
]

bounds = optimize.Bounds(lower_bounds, upper_bounds)

In [ ]:
R_nlo = nlo[:, 0]
x_nlo = const.BETA4 / R_nlo
energies_nlo = -nlo[:, 1:].T

ii = np.intersect1d(
    np.where(x_nlo > 60)[0],
    np.where(x_nlo < 70)[0]
)

fig, ax = plt.subplots(dpi=200)
fig.patch.set_facecolor('white')

nlo_asymp = []

first = True
for row in energies_nlo[:, ii]:
#     if first:
#         guess = [row[-1], -0.1]
#     if np.any(row > 0):
#         row *= -1
#     pars, cov = optimize.curve_fit(convergence2, x_nlo[ii], row, p0=guess, maxfev=10000)
    
#     if first:
#         guess = [row[-1], -0.01, 1]
    if np.any(row > 0):
        row *= -1
    
    converged = False
    while not converged:
        guess = [row[-1], 10*np.random.rand()-5, 100*np.random.rand() - 50]
        pars, cov, infodict, msg, ier = optimize.curve_fit(
            lambda x, c1, c2, c3: convergence12_improved(x, c1, c2, c3, gamma), 
            x_nlo[ii], row, p0=guess, maxfev=100000, bounds=bounds, full_output=True,
            sigma = 0.1*np.abs(np.mean(row))*np.ones(row.size)
        )
        err = np.sqrt(np.sum(np.array(infodict['fvec'])**2))
        converged = err < 1
        print(guess, err)
        os.system('clear')
    
    first = False
    sigma = np.sqrt(np.diag(cov))
    guess = pars
#     print(f'{row[-1]:.2e} | {pars[0]:2e} ± {sigma[0]:.2e}')
    print(pars, '\n')
    
    nlo_asymp.append(pars[0])
    ax.plot(x_nlo[ii], np.abs(row))
    
    xp = np.linspace(x_nlo[ii][0], 5*x_nlo[ii][-1], 100)
    ax.plot(xp, np.abs(convergence12_improved(xp, *pars, gamma)), linestyle='--')
    
    ax.axhline(np.abs(pars[0]), color='C6')
    
ax.set_xlabel(r'$\beta_4/R$')
ax.set_ylabel(r'$E_n$')
# ax.set_ylim([1e-5, 2e-3])
ax.set_yscale('log');

## NLO Linear and Quadratic

In [ ]:
R_nlo = nlo[:, 0]
x_nlo = const.BETA4 / R_nlo
energies_nlo = -nlo[:, 1:].T

ii = np.intersect1d(
    np.where(x_nlo > 50)[0],
    np.where(x_nlo < 70)[0]
)

fig, ax = plt.subplots(dpi=200)
fig.patch.set_facecolor('white')

nlo_asymp = []

first = True
for (i, row) in enumerate(energies_nlo[:, ii]):
    if first:
        guess = [row[-1], -0.1, 1]
    if np.any(row > 0):
        row *= -1
        if first:
            guess = [row[-1], 0.1, -1]
    pars, cov = optimize.curve_fit(convergence12, x_nlo[ii], row, p0=guess, maxfev=100000)
    
    first = False
    sigma = np.sqrt(np.diag(cov))
    guess = pars
#     print(f'{row[-1]:.2e} | {pars[0]:2e} ± {sigma[0]:.2e}')
    print(pars, '\n')
    
    nlo_asymp.append(pars[0])
    ax.plot(x_nlo[ii], np.abs(row), color=f'C{i}')
    ax.plot(x_nlo[ii], np.abs(convergence12(x_nlo[ii], *pars)), color=f'C{i}', linestyle='--')
    ax.axhline(np.abs(pars[0]), color=f'C{i}')
   
ax.set_yscale('log')
ax.set_xlabel(r'$\beta_4/R$')
ax.set_ylabel(r'$B_n$');

In [ ]:
a

In [ ]:
def read_slope(x, y):
    x1, x2 = x[[-1, 0]]
    y1, y2 = y[[-1, 0]]
    print(x1, y1)
    print(x2, y2)
    m = (y2-y1) / (x2-x1)
    b = y2 - m*x2
    return m, b

In [ ]:
lo_asymp / mm - 1

In [ ]:
nlo_asymp[:-1] / mm[:-1] - 1

In [ ]:
pars_lo = read_slope(np.log(2*const.MU*(-mm)*const.BETA4**2),
                     np.log(np.abs(np.array(lo_asymp) / mm - 1)))
pars_nlo = read_slope(np.log(2*const.MU*(-mm[:-1])*const.BETA4**2),
                      np.log(np.abs(np.array(nlo_asymp[:-1]) / mm[:-1] - 1)))

In [ ]:
pars_nlo[0] / pars_lo[0]

In [ ]:
xs = np.linspace(60, 3e4, 1000)

In [ ]:
pars_lo, pars_nlo

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=300)
fig.patch.set_facecolor('white')

ax.scatter(2*const.MU*(-mm)*const.BETA4**2, np.abs(-np.array(lo_asymp) / -mm - 1))
ax.scatter(2*const.MU*(-mm[3:-1])*const.BETA4**2, np.abs(-np.array(nlo_asymp[3:-1]) / -mm[3:-1] - 1), marker='s')
ax.scatter(2*const.MU*(-mm[:-4])*const.BETA4**2, np.abs(-np.array(nlo_asymp[:-4]) / -mm[:-4] - 1), marker='x',
           color='C1')

# ax.plot(xs, pars_lo[0]*xs + pars_lo[1])
# ax.plot(np.exp(xs), pars_nlo[0]*xs + pars_nlo[1])

ax.set_xlabel(r'$[\beta_4/\beta^{(n)}_{\rm TL}]^2$')
ax.set_ylabel(r'$|B^{(n)}/B^{(n)}_{\rm TL} - 1|$')

ax.set_xscale('log')
ax.set_yscale('log');

plt.savefig('figures/lo_nlo_error.pdf')

# ax.set_xlim([]);

In [ ]:
for (lo, nlo, x) in zip(lo_asymp[::-1], nlo_asymp[::-1], mm[::-1]):
    print(f'{-lo:.2e} & {-nlo:.2e} & {-x:.2e} \\\\')
    print('\\hline')

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(2*4, 4*3), dpi=200)
fig.patch.set_facecolor('white')
fig.delaxes(ax[3, 1])

num_pts = 10
stop = np.argmin(np.abs(const.BETA4/nlo[:, 0] - 70)) + 1

for k in range(7):
    i = k//2
    j = k%2
#     ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], lo[stop-num_pts:stop, k+1], label=r'$E^{({\rm LO})}(R)$')
    ax[i, j].plot(const.BETA4/nlo[stop-num_pts:stop, 0], nlo[stop-num_pts:stop, k+1],
                  label=r'$E^{({\rm NLO})}(R)$', marker='o', linestyle='')
#     ax[i, j].axhline(mm[k], linestyle='--', color='C6', label='MM')
    
    par, cov = optimize.curve_fit(convergence, const.BETA4/lo[stop-num_pts:stop, 0], lo[stop-num_pts:stop, k+1])
#     ax[i, j].axhline(par[0], color='C0', linestyle='--', label=r'$E_\infty^{({\rm LO})}$')
#     ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], convergence(const.BETA4/lo[stop-num_pts:stop, 0], *par),
#                   linestyle='--', label='LO fit')
    lo_asymp[k] = par[0]
    
    par, cov = optimize.curve_fit(convergence12, 
                                  const.BETA4/nlo[stop-num_pts:stop, 0], 
                                  nlo[stop-num_pts:stop, k+1],
                                  p0=[1, 1, 1],
                                  maxfev=3000)
#     ax[i, j].axhline(par[0], color='C1', linestyle='--', label=r'$E_\infty^{({\rm NLO})}$')
    y_nlo = nlo[stop-num_pts:stop, k+1]
    mu_nlo = convergence12(const.BETA4/nlo[stop-num_pts:stop, 0], *par)
    print(f'{np.sum((y_nlo - mu_nlo)**2) / (num_pts - par.size):.2e}')
    ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], mu_nlo,
                  linestyle='--', label='NLO fit')
    nlo_asymp[k] = par[0]
    
    ax[i, j].set_ylabel(r'$E_{n=%d}$' % (k))
    ax[i, j].set_xlabel(r'$\beta_4/R$')
    if k == 6:
        ax[i, j].legend()
    
#     f = interp1d(const.BETA4/lo[-num_pts:, 0], lo[-num_pts:, k+1], kind='cubic', fill_value='extrapolate')
#     ax[i, j].scatter(50, f(50))
    
#     g = interp1d(const.BETA4/nlo[-num_pts:, 0], nlo[-num_pts:, k+1], kind='cubic', fill_value='extrapolate')
#     ax[i, j].scatter(50, g(50))
    

$$\sum_i (y_i - \mu_i)^2 / \nu $$

| Quadratic | Both |
| --------- | ---- |
| 7.91e-07  | 6.01e-07 |
| 6.15e-08  | 6.29e-08 |
| 3.97e-08  | 1.94e-08 |
| 1.12e-08  | 2.17e-09 |
| 4.38e-10  | 5.35e-11 |
| 2.39e-12  | 2.24e-13 |
| 2.77e-26  | 9.01e-27 |

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=300)
fig.patch.set_facecolor('white')

num_pts = 34
stop = np.argmin(np.abs(const.BETA4/lo[:, 0] - 30)) + 1

for k in range(7):
    i = k//2
    j = k%2
    ax.plot(const.BETA4/lo[stop-num_pts:stop, 0], -lo[stop-num_pts:stop, k+1], label=r'$E^{({\rm LO})}(R)$',
            color='C0')
    ax.plot(const.BETA4/nlo[stop-num_pts:stop, 0], -nlo[stop-num_pts:stop, k+1], label=r'$E^{({\rm NLO})}(R)$',
            color='C1', linestyle='-.')
    ax.axhline(-mm[k], linestyle='--', color='C6', label='MM')
    ax.set_yscale('log')
#     par, cov = optimize.curve_fit(convergence, const.BETA4/lo[stop-num_pts:stop, 0], lo[stop-num_pts:stop, k+1])
#     ax[i, j].axhline(par[0], color='C0', linestyle='--', label=r'$E_\infty^{({\rm LO})}$')
#     ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], convergence(const.BETA4/lo[stop-num_pts:stop, 0], *par),
#                   linestyle='--', label='LO fit')
#     lo_asymp[k] = par[0]
    
#     par, cov = optimize.curve_fit(convergence2, const.BETA4/nlo[stop-num_pts:stop, 0], nlo[stop-num_pts:stop, k+1])
#     ax[i, j].axhline(par[0], color='C1', linestyle='--', label=r'$E_\infty^{({\rm NLO})}$')
#     ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], convergence2(const.BETA4/lo[stop-num_pts:stop, 0], *par),
#                   linestyle='--', label='NLO fit')
#     nlo_asymp[k] = par[0]
    
    ax.set_ylabel(r'$B_{n}$')
    ax.set_xlabel(r'$\beta_4/R$ (a.u.)')
#     if k == 6:
#         ax[i, j].legend()
    
#     f = interp1d(const.BETA4/lo[-num_pts:, 0], lo[-num_pts:, k+1], kind='cubic', fill_value='extrapolate')
#     ax[i, j].scatter(50, f(50))
    
#     g = interp1d(const.BETA4/nlo[-num_pts:, 0], nlo[-num_pts:, k+1], kind='cubic', fill_value='extrapolate')
#     ax[i, j].scatter(50, g(50))

plt.savefig('figures/lo_nlo_spectrum_R.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3), dpi=300)
fig.patch.set_facecolor('white')

num_pts = 34
stop = np.argmin(np.abs(const.BETA4/lo[:, 0] - 30)) + 1

for k in range(7):
    i = k//2
    j = k%2
    ax.plot(const.BETA4/lo[stop-num_pts:stop, 0], -lo[stop-num_pts:stop, k+1], label=r'$E^{({\rm LO})}(R)$',
            color='C0')
    ax.plot(const.BETA4/nlo[stop-num_pts:stop, 0], -nlo[stop-num_pts:stop, k+1], label=r'$E^{({\rm NLO})}(R)$',
            color='C1', linestyle='-.')
    ax.axhline(-mm[k], linestyle='--', color='C6', label='MM')
    ax.set_yscale('log')
#     par, cov = optimize.curve_fit(convergence, const.BETA4/lo[stop-num_pts:stop, 0], lo[stop-num_pts:stop, k+1])
#     ax[i, j].axhline(par[0], color='C0', linestyle='--', label=r'$E_\infty^{({\rm LO})}$')
#     ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], convergence(const.BETA4/lo[stop-num_pts:stop, 0], *par),
#                   linestyle='--', label='LO fit')
#     lo_asymp[k] = par[0]
    
#     par, cov = optimize.curve_fit(convergence2, const.BETA4/nlo[stop-num_pts:stop, 0], nlo[stop-num_pts:stop, k+1])
#     ax[i, j].axhline(par[0], color='C1', linestyle='--', label=r'$E_\infty^{({\rm NLO})}$')
#     ax[i, j].plot(const.BETA4/lo[stop-num_pts:stop, 0], convergence2(const.BETA4/lo[stop-num_pts:stop, 0], *par),
#                   linestyle='--', label='NLO fit')
#     nlo_asymp[k] = par[0]
    
    ax.set_ylabel(r'$B_{n}$')
    ax.set_xlabel(r'$\beta_4/R$ (a.u.)')
#     if k == 6:
#         ax[i, j].legend()
    
#     f = interp1d(const.BETA4/lo[-num_pts:, 0], lo[-num_pts:, k+1], kind='cubic', fill_value='extrapolate')
#     ax[i, j].scatter(50, f(50))
    
#     g = interp1d(const.BETA4/nlo[-num_pts:, 0], nlo[-num_pts:, k+1], kind='cubic', fill_value='extrapolate')
#     ax[i, j].scatter(50, g(50))

# plt.savefig('figures/lo_nlo_spectrum_R.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=200)
fig.patch.set_facecolor('white')

ax.scatter(lo_asymp, nlo_asymp - lo_asymp)

ax.set_xlabel(r'$E_{\rm LO}$')
ax.set_ylabel(r'$\Delta_{\rm NLO}$');

In [ ]:
for (e_lo, e_nlo, e_mm) in zip(lo_asymp[::-1], nlo_asymp[::-1], mm[::-1]):
    print(r'%.2e & %.2e & %.2e \\' % (-e_mm, -e_lo, -e_nlo))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=200)
fig.patch.set_facecolor('white')

ax.scatter(mm, (mm - lo_asymp)/mm)
ax.scatter(mm, (mm - nlo_asymp)/mm)
# ax.axvline(-1/(2*const.MU*5**2))

ax.set_xlabel(r'$E_{\rm MM}$')
ax.set_ylabel(r'$(E_{\rm MM} - E_{\rm id})/E_{\rm MM}$');

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=300)
fig.patch.set_facecolor('white')

ax.plot(np.abs(mm), np.abs((mm - lo_asymp)/mm), marker='o', linestyle='', label='LO')
# ax.loglog(np.abs(mm[:-1]), np.abs((mm[:-1] - nlo_asymp[:-1])/mm[:-1]), marker='o', linestyle='', label='NLO')
# ax.axvline(-1/(2*const.MU*5**2))

# ax.legend()
ax.set_xlabel(r'$B_{\rm MM}$')
ax.set_ylabel(r'$\left( B_{\rm MM} - B_{\rm id} \right)/B_{\rm MM}$');

plt.savefig('figures/lo_error.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=200)
fig.patch.set_facecolor('white')

ax.loglog(np.abs(mm), np.abs((mm - lo_asymp)/mm), marker='o', linestyle='', label='LO')
ax.loglog(np.abs(mm[:-1]), np.abs((mm[:-1] - nlo_asymp[:-1])/mm[:-1]), marker='o', linestyle='', label='NLO')
# ax.axvline(-1/(2*const.MU*5**2))

ax.legend()
ax.set_xlabel(r'$B_{\rm MM}$')
ax.set_ylabel(r'$(B_{\rm MM} - B_{\rm id})/B_{\rm MM}$');

In [ ]:
par_lo, cov_lo = optimize.curve_fit(lambda x, m, b: m*x+b,
                                    np.log(np.abs(mm)),
                                    np.log(np.abs((mm - lo_asymp)/mm)))
par_lo

In [ ]:
par_nlo, cov_nlo = optimize.curve_fit(lambda x, m, b: m*x+b,
                              np.log(np.abs(mm[2:-1])),
                              np.log(np.abs((mm[2:-1] - nlo_asymp[2:-1])/mm[2:-1]))
                             )
par_nlo

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=300)
fig.patch.set_facecolor('white')

ax.scatter(np.abs(mm), np.abs((mm - lo_asymp)/mm), marker='o', label='LO')
ax.scatter(np.abs(mm[:-1]), np.abs((mm[:-1] - nlo_asymp[:-1])/mm[:-1]), marker='s', label='NLO')
# ax.axvline(-1/(2*const.MU*5**2))

xs = np.linspace(np.log(np.abs(mm)[0]), np.log(np.abs(mm)[-1]), 100)
ax.plot(np.exp(xs), np.exp(par_lo[0]*xs + par_lo[1]), color='C0', linestyle='-')
ax.plot(np.exp(xs), np.exp(par_nlo[0]*xs + par_nlo[1]), color='C1', linestyle='-.')

ax.legend()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$B_{\rm TL}$')
ax.set_ylabel(r'$(B_{\rm TL} - B_{\rm id})/B_{\rm TL}$');

plt.savefig('figures/lo_nlo_error_loglog.pdf')